# Predicting the past, United States

## Introduction

In the United States, 112 commercial reactors have been deployed and operated since 1967. Predicting the past repository uses published data of those commercial reactors and simulates these reactors using _CYCLUS_, an agent-based simulation software, to predict and compare information such as the amount of power generated. Furthermore, these simulations can be used as baseline comparison models for any future studies that involve simulations modelling or predicting the future such is as DOE-NE's study to screen fuel cycle options that aims to identify the potential benefits and challegnes of fuel cycle options.


## Obtaining Recipes for Simulation

First, a input file for _CYCLUS_ that specifies information such as the simulation start year, reactor deployment year, facility deployment, and recipes for fresh and spent nuclear fuel (SNF) need to be produced. The recipes for fresh and SNF were obtained from `vision_recipes.xls` spreadsheet file. The file contains composition data for fresh and spent nuclear fuel for uranium oxide (UOX) and mixed oxide (MOX) fuels for different burn ups.

Importing raw composition data from `vision_recipes.xls` was performed through python and will be demonstrated below. Due to limited information regarding the reactors, all reactors were assumed to operate at a burn up of 51 GWd/MTHM.

### Demonstration
__Import necessary libraries__

In [ ]:
import csv
import jinja2
import pyne.nucname as nn

__Read fuel recipe data__

Opens and saves contents from `vision_recipes.xls` files to a list by looping over each cell in the spreadsheet file.

In [ ]:
def import_csv(in_csv, delimit):
    """ Imports contents of a comma delimited csv file
    to a 2D list.

    Parameters
    ---------
    in_csv: str
        csv file name.
    delimit: str
        delimiter of the csv file

    Returns
    -------
    data_list: list
        list with fleetcomp data.
    """
    with open(in_csv, encoding='utf-8') as source:
        sourcereader = csv.reader(source, delimiter=delimit)
        data_list = []
        for row in sourcereader:
            data_list.append(row)
    return data_list

recipes = import_csv('import_data/vision_recipes/uox.csv', ',')
recipes

__Load template for _CYCLUS_ input file__

Imports the template file used to render recipe data.

In [ ]:
def load_template(in_template):
    """ Returns a jinja2 template.

    Parameters
    ---------
    in_template: str
        template file name.

    Returns
    -------
    output_template: jinja template object
    """
    with open(in_template, 'r') as default:
        output_template = jinja2.Template(default.read())
    return output_template

recipe_template = load_template('templates/US/recipes_template.xml')
recipe_template

__Select appropriate recipe for a given burnup__

Uses the imported list and a given amount of burnup to select the appropriate composition data. The data is then stored into a dictionary.

In [ ]:
def get_composition(in_list, burnup):
    """ Returns a dictionary of reactor name and build_time (in months)
    using the fleetcomp list for reactors specified in *args.

    Parameters
    ---------
    in_list: list
        list file containing fleetcomp data.
    *args: str
        path and name of reactors that will be added to cyclus simulation.

    Returns
    -------
    data_dict: dictionary
        dictionary with key: isotope, and value: composition.
    """
    data_dict = {}
    for i in range(len(in_list)):
        if i > 1:
            if burnup == 33:
                data_dict.update({nn.id(in_list[i][0]):
                                  float(in_list[i][2])})
            elif burnup == 51:
                data_dict.update({nn.id(in_list[i][0]):
                                  float(in_list[i][4])})
            else:
                data_dict.update({nn.id(in_list[i][0]):
                                  float(in_list[i][6])})
    return data_dict

composition = get_composition(recipes, 51)
composition

__Render recipe template with recipe data__

Uses `jinja2` to render recipe data stored in a dictionary to `recipe_template` and `pyne` to convert isotope name for _CYCLUS_. Saves rendered object file to an xml file for use with _CYCLUS_.

In [ ]:
def write_recipes(in_dict, in_template, burnup):
    """ Renders jinja template using data from in_list and
    outputs an xml file for a single reactor.

    Parameters
    ---------
    data_dict: dictionary
        dictionary with key: isotope, and value: composition.
    in_template: jinja template object
        jinja template object to be rendered.

    Returns
    -------
    null
        generates reactor files for cyclus.
    """
    rendered = in_template.render(vision=in_dict)
    with open('cyclus/input/US/recipes/uox_' + str(burnup) +
              '.xml', 'w') as output:
        output.write(rendered)

write_recipes(composition, recipe_template, 51)

[uox_51.xml](cyclus/input/US/recipes/uox_51.xml) file shows the rendered result. 

__recipe template__

`jinja2` allows iteration over iterable objects such as lists, sets, and dictionaries. This allows end-users to produce rendered output files that vary in length depending on the data that it has been rendered with. An example is shown below.

Recipe template file:

```
<recipes>
  <recipe>
    <name>fresh_uox</name>
    <basis>atom</basis>
    <nuclide>
      <id>922350000</id>
      <comp>4.5</comp>
    </nuclide>
    <nuclide>
      <id>922380000</id>
      <comp>95.5</comp>
    </nuclide>
  </recipe>
  <recipe>
    <name>spent_uox</name>
    <basis>atom</basis>
    {% for key, value in vision.items() -%}
    <nuclide>  <id>{{ key }}</id>  <comp>{{ value }}</comp>  </nuclide>
    {% endfor -%}
  </recipe>
</recipes>
```

The template is a template for an xml file that follows the _CYCLUS_ xml schema. `jinja2` recognizes the for-loop in the following lines 
```
{% for key, value in vision.items() -%}`
<nuclide>  <id>{{ key }}</id>  <comp>{{ value }}</comp>  </nuclide>
{% endfor -%}
```
and iterates over the composition dictionary to render the isotope and its composition.

[uox_51.xml](cyclus/input/US/recipes/uox_51.xml) file shows the rendered result. 

### Obtaining Deployment Data

Reactors specified in [US fleetcomp](import_data/fleetcomp/US_Fleet.txt) spreadsheet, need to be properly imported and deployed at the correct timesteps for a successful simulation.

   
### Adding Deployment Data
#### Files Used

+ ##### Input
    *  [US_Fleet.txt](import_data/fleetcomp/US_Fleet.txt) - fleetcomp data of US nuclear power plants.
    *  [deployinst_template.xml](templates/deployinst_template.xml) - an XML template that provides the structure for _CYCAMORE_ _DEPLOYISNT_ archetype.
    *  [inclusions_template.xml](templates/inclusions_template.xml) - an XML file used to declare all XML inclusion links using xinclude. The referenced files contain information regarding reactors such as core mass, power_cap, etc.
    *  [112 Reactor files](cyclus/input/reactors) - 112 files containing information regarding reactors deployed in the United States since 1965.

+ ##### Script
    *  [import_data.py](import_data.py) - a script that is used to import, process and produce recipes for UOX, and SNF.

+ ##### Output
    *  [deployinst.xml](cyclus/input/buildtimes/deployinst.xml) - an XML file containing deployment information regarding all reactors that are deployed in the _CYCAMORE DEPLOYINST_ archetype.
    *  [inclusions.xml](cyclus/input/buildtimes/inclusions.xml) - an XML file that has xinclude links for _CYCLUS_ input file. The referenced files include information regarding reactors.

    This process involves two main steps. Calculating deployment time steps for _CYCAMORE DEPLOYISNT_ archetype, and using `jinja` templates to make two XML files: `deployinst` and `inclusions`. By default, the `import_data` script deploys all reactors contained in the US fleetcomp.
    
    The buildtimes for all reactors were obtained from the `US_Fleet` text file, which contains the deployment date of the reactors. By comparing and finding the difference between the simulation start date and the reactor deployment date, an appropriate build_time was obtained in units of months. Note that since _CYCAMORE DEPLOYINST_ archetype only accepts integer values for reactor build time, the final difference is casted into an integer value within python. In addition, the following assumptions were made in calculating the difference between simulation start date and reactor deployment date,

+ ###### Assumptions
    * Differences were calculated individually in YEARS, MONTHS, and DAYS
    * The difference in DAYS were converted into MONTHS by using the ratio between the number of days in a year (365 DAYS) and the number of months in year (12 MONTHS) so that each month is assumed to have the same number of days.
    * The difference in YEARS were converted into MONTHS by using the number of months in a year (12 MONTHS).
    
    Then a dictionary of the reactor name (key), and build time (value) was passed to a function that renders the information contained in the dictionary to the output files listed above.

### Setting XML base for final input file
#### Files Used

+ ##### Input
    *  [predicting_the_past_template.xml](templates/predicting_the_past_template.xml) - an XML template including a _CYCLUS_ input file.

+ ##### Script
    *  [import_data.py](import_data.py) - a script that is used to import, process and produce recipes for UOX, and SNF

+ ##### Output
    *  [predicting_the_past.xml](cyclus/input/predicting_the_past.xml) - Final input file for _CYCLUS_

    One of the issues associated with using xinclude to include external XML files referenced by a relative path is that the path is sensitive the directory to which it is called. For instance, if the final _CYCLUS_ input file was called in any other directory than `cyclus/input`, then XML parsers, such as the one used in _CYCLUS_ is unable to find the files specified in the input file. To fix this issue, a `jinja` template of the _CYCLUS_ input file is created and rendered with the absolute directory of the _CYCLUS_ input file. This allows XML parsers to recognize the relative paths used in xinclude as paths relative to that specified as `xml base`. This is also done in the `import_data` script.

### Running Cyclus simulation
#### Files Used

+ ##### Input
    *  [predicting_the_past.xml](cyclus/input/predicting_the_past.xml) - Final input file for _CYCLUS_

+ ##### Output
    *  [US.sqlite](cyclus/US.sqlite) - A _CYCLUS_ simulation result file.

    _CYCLUS_ simulation is run. This is often performed in a separate ipython notebook ([Analysis.ipynb](analysis/Analysis.ipynb)). To view the output file from a fresh copy of the Github repository, please open the ipython notebook above to run the _CYCLUS_ simulation with the input file above.
    
### Analyzing Output
#### Files Used

+ ##### Input
    *  [US.sqlite](cyclus/US.sqlite) - A _CYCLUS_ simulation result file.
    
+ ##### Script
    *  [analysis.py](analysis/analysis.py) - A python script containing functions required for the analysis of _CYCLUS_ output file
    
    The analysis for the _CYCLUS_ simulation is done using the `analysis` script and is outlined in a separate ipython notebook ([Analysis.ipynb](analysis/Analysis.ipynb)). A number of different data was obtained from the _CYCLUS_ output file. The following data was obtained from the _CYCLUS_ output file. 
    
    
        1. Amount of natural uranium consumed over time
        2. Amount of fuel into all reactors over time
        3. Fuel utilization over time
        4. Capacity over time
        5. Number of reactors over time
        6. Separative Work Unit (SWU) over time
        7. Amount of Spend Nuclear Fuel (SNF) in Sink over time
        8. Amount of tailings over time
        9. Power generated over time
        
    1. Amount of natural uranium consumed over time
    
    This uses `nat_u_timeseries` function within `analysis` script that makes a simple sqlite query to the `timeseries enrichmentfeed` table in the _CYCLUS_ output file. The query requests the the amount of uranium sent to any enrichment facilities at each time step and passes those values into `get_timeseries_cum` function. 
    
    The `get_timeseries_cum` function accepts a list that contains time (in index 0) and value (in index 1), and calculates the cumulative sum of those values in chronological order. This new list is then returned to plot the values in a graph, which is shown in the Results section.
    
    2. Amount of fuel into all reactors over time
    
    This uses `fuel_into_reactors` function within `analysis` script. The `fuel_into_reactors` function makes an sqlite query from `transactions`, `resources`, and `agententry` table to search for the amount of fuel sent to any reactors at each time step. The function utilizes `INNER JOIN` capabilities of sqlite3 to merge the three tables above. It searches for matching resources in transactions done betweem enricment facilities and reactors to find records of fuel sent to reactors. The results of this query is passed to the `get_timeseries_cum` function to obtain a cumulative list of fuel sent to reactors over time, which is then used to plot the values into a graph.
    
    3. Fuel utilization over time
    
    This uses `u_util_calc` function within `analysis` script that calls `nat_u_timeseries` and `fuel_into_reactors` functions and saves the resulting timeseries list into a numpy array. Then the function performs an element-by-element division of fuel_timeseries by uranium_supply_timeseries. The resulting array contains the timeseries list of fuel utilization factor. The resulting timeseries list is used for plotting.
    
    4. Capacity over time
    
    This uses `get_power_dict` function within `analysis` script. This function makes two function calls and two sqlite queries. First, `get_timesteps` function, and `get_inst` function are called. The former returns the starting year, month, duration, and a numpy linspace that contains the timesteps of the simulation. The latter returns the prototype and the agent ids of any institutions specified in the _CYCLUS_ simulation. The first sqlite query obtains the agentid, entrytime, power, and parentid. The second sqlite query obtains the agentid, exittime, power, and parentid. Then, all information is passed into `capacity_calc` function. The `capacity_calc` function collects these input parameters, then loops through each time step, and calculates the cumulative capacity of all the reactors for each _CYCLUS_ institution.
    
    5. Number of reactors over time
    
    This uses `get_deployment_dict` function within `analysis` script. This function is very similar to `get_power_dict` in that the same function calls and queries are made. The results of these function calls and queries are passed to `reactor_deployments`, which instead of producing a timeseries of capacity, produces a timeseries of reactors deployed.
    
    6. SWU over time
    
    This uses `get_swu_dict` function within `analysis` script. The function first gets the agent_id of all enrichment facilities in the simulation. Then, it loops through the list of enrichment agent_ids and performs the following: make an sqlite query to obtain the SWU at each time step for an enrichment facility, passes the result of the query to `get_timeseries_cum` to obtain the timeseries of the cumulative SWU, and stores the timeseries list into a dictionary value with the enrichment facilities agent id as key. This dictionary is used to produce a graph of SWU over time for each enrichment facility.
    
    7. Amount of SNF in sink over time
    
    This uses `facility_commodity_flux` function within `analysis` script. The function takes in a sqlite cursor, list of agent_ids, list of commodities, and a boolean value to specify influx or out flux. It loops through the list of commodities, and makes an sqlite query to obtain the amount of commodity transferred at each time step. Then, it passes the results of the query to `get_timeseries_cum` to obtain the timeseries list of each commodity. The lists are then stored as dictionary values with the commodity as key.
    
    8. Amount of tailings over time
    
    This also uses `facility_commodity_flux` function within `analysis` script. First the function makes an sqlite query to obtain the agent id for a  Low Level Waste (LLW) sink facility and stores the agent ids in a list. Then it calls `facility_commodity_flux` with tails as the commodity parameter. The resulting dictionary was used to plot the timeseries of tailings over time.
    
    9. Power generated over time
    
    This uses `get_power_dict` function, and capacity factor information published by the NEI (US Nuclear Generating Statistics). The power generated is calculated with the following assumptions.
   
        1.  Capacity factor is constant for every month in a year
        2.  For years where the capacity factor data is missing, the capacity factor is assumed constant
        3.  Refueling time for all reactors is 1 month
        
    The power generated is calculated by calculating the element-by-element product of capacity factor and overall capacity then converting units from giga-watts (GW) to giga-watt-hours (GWh). The resulting dictionary is used to plot the power generated over time. 
    
    A similar plot of power generated was produced from the data published by the NEI (US Nuclear Generating Statistics). The data contains the mount of power generated each year in mega-watt-hours (MWh). The plot was generated using one of the plotting functions in the `analysis` script. Since ,the plotting function requires power generated at each month, rather than year, the power generated was assumed to be constant throughout the months in a year.

## Results

The results for the analysis performed above are shown below. Unfortunately, the only data that is available for comparison is the power generated over time. NEI has also published this on their website (US Nuclear Generating Statistics). The data published by the NEI was saved in a spreadsheet so that it can be plotted for comparison. Since the data contains power generated by US nuclear power plants from 1971 to 2016, a new plot was generated to only show the power generated between that period in Figure 10. The plot of power generated produced from the data published by the NEI is shown in Figure 11.

1. Amount of natural uranium consumed over time
![Nat_u vs. Time](analysis/results/US/Nat_u consumption.png)
2. Amount of fuel into all reactors over time
![Fuel to Reactors vs. Time](analysis/results/US/Fuel to Reactors over Time.png)
3. Fuel utilization over time
![Fuel utilization vs. Time](analysis/results/US/Fuel utilization.png)
4. Capacity over time
![Capacity vs. Time](analysis/results/US/Capacity vs Time.png)
5. Number of reactors over time
![Number of Reactors vs. Time](analysis/results/US/Number of Reactors vs Time.png)
6. Separative Work Unit (SWU) over time
![SWU vs. Time](analysis/results/US/SWU vs Time.png)
7. Amount of Spend Nuclear Fuel (SNF) in Sink over time
![Spent_UOX in Sink vs. Time](analysis/results/US/Spent_UOX in Repository.png)
8. Amount of tailings over time
![Tailings vs. Time](analysis/results/US/Tailings in Sink.png)
9. Power generated over time
![Power Generated vs. Time](analysis/results/US/Power Generated.png)
10. Power generated between 1971~2016
![Power Generated (1971~2016)](analysis/results/US/Power Generated 1971~2016.png)
11. Power generated between 1971~2016 (US Nuclear Generating Statistics, NEI)
![Power Generated NEI (1971~2016)](analysis/published_data/US/Power Generated NEI.png)

## Discussion

As shown in Figure 10 and 11, the plot of the amount of power generated between 1971 and 2016 from the _CYCLUS_ simulation matches very closely to that published by the NEI. There are a few differences that can be noticed between the two plots. First, the power generated according to _CYCLUS_ is slightly higher than the power generated according to the NEI. This can be explained from the difference in refueling times between simulation and real-world scenarios. Due to software limitations of _CYCLUS_, refueling time for reactors can only be entered in integers of month, and stays constant throughout the lifetime of the plant whereas actual refueling time can vary in floating points of month and changes depending on various factors.

_CYCLUS_ simulation used to generate Figure 10 assumes a refueling period of 1 month. According to the NEI, the average  refueling period for reactors in the United States varies quite significantly. The average refueling period in 1990 was 104 days, a period of over 3 months, and generally decreases to an average refueling period of 35 days, a period of just over 1 month, in 2017. As published by the NEI, the average refueling period of nuclear reactors in the United States has been greater than 1 month between 1990 and 2017. This means that the power generated by the reactors deployed in the _CYCLUS_ simulation would have a greater amount of power generated over a period of time.

## Future Goals
+  Specify actual burnup for reactors
    + Currently, the reactors deployed in Cyclus are all deployed with a burnup of 51 GWd/MTHM. 
    
    
+  Specify reactor n_assembly_core and batch
    + Reactors were deployed in cyclus without the actual number of assemblies per core and batch. The typical values were used for each reactor type (Source needs to be added for the typical values). While the effects of changing these values may not be significant in the outcome of the simulation, specifying such information may benefit the accuracy of the simulation results.
    
    
+  Apply a similar analysis to different regions such as the EU or in the world


## References
*  US Nuclear Generating Statistics. (n.d.). Retrieved from https://www.nei.org/Knowledge-Center/Nuclear-Statistics/US-Nuclear-Power-Plants/US-Nuclear-Generating-Statistics


*  US Nuclear Refueling Outage Days. (n.d.). Retrieved from https://www.nei.org/Knowledge-Center/Nuclear-Statistics/US-Nuclear-Power-Plants/US-Nuclear-Refueling-Outage-Days


## Additional Information

### XInclude

_CYCLUS_'s XML parser has the capabilities of parsing xinclude. To include an XML file (i.e. a.xml) into another XML (i.e. b.xml) file, add xinclude in the XML namespace by adding the following to the root tag of `b.xml`:

> xmlns:xi="http://www.w3.org/2001/XInclude"

Then, reference the XML file to be added (`a.xml`) into the appropriate level with the following line:

> `<xi:include href="[path_to_a.xml/a.xml]" />`

The path specified above can be absolute or relative. However, if it is relative, the path where the XML parsing is called needs to be the same as the path of the file that has all the XML inclusions. One way to avoid this issue with xinclude is to add an `xml:base` tag that contains the absolute path of the file that has all the XML inclusions.